In [94]:
import pandas as pd
#importamos las librerias y cargamos los datasets
am = pd.read_csv('Data/amazon_prime_titles.csv', index_col=False)
di = pd.read_csv('Data/disney_plus_titles.csv', index_col=False)
hu = pd.read_csv('Data/hulu_titles.csv', index_col=False)
ne = pd.read_csv('Data/netflix_titles.csv', index_col=False)

## Generaremmos una campo ID, con la primera letra de la platadorma seguido del show_id como valor

In [95]:
#Creamos la columna ID con la funcion insert(posicion, nombre de columna, valor.tipo de dato)
am.insert(0, 'ID', 'a' + am['show_id'].astype(str))
di.insert(0, 'ID', 'd' + di['show_id'].astype(str))
hu.insert(0, 'ID', 'h' + hu['show_id'].astype(str))
ne.insert(0, 'ID', 'n' + ne['show_id'].astype(str))


In [96]:
#Verificamos
am.head()

,ID,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,as1,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...
1,as2,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...
2,as3,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...
3,as4,s4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,"March 30, 2021",2014,NaN,69 min,Documentary,"Pink breaks the mold once again, bringing her ..."
4,as5,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,"March 30, 2021",1989,NaN,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...


In [97]:
am.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9668 entries, 0 to 9667
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ID            9668 non-null   object
 1   show_id       9668 non-null   object
 2   type          9668 non-null   object
 3   title         9668 non-null   object
 4   director      7586 non-null   object
 5   cast          8435 non-null   object
 6   country       672 non-null    object
 7   date_added    155 non-null    object
 8   release_year  9668 non-null   int64 
 9   rating        9331 non-null   object
 10  duration      9668 non-null   object
 11  listed_in     9668 non-null   object
 12  description   9668 non-null   object
dtypes: int64(1), object(12)
memory usage: 982.0+ KB


## Primero vamos a cargar los datos faltantes y unirlos

In [98]:
#Creamos una lista para ingestar los 8 archivos en un solo dataset
df_list = []

import glob

#Con la libreria glob y la funcion glob.glob() importamos todos los archivos tipo .csv para luego agregarlos en la lista
for file in glob.glob('Data/ratings/ratings/*.csv'):
    df = pd.read_csv(file)
    df_list.append(df)

In [99]:
#Concatenamos todos los DF de la lista
rati = pd.concat(df_list)

In [100]:
#cambiamos la columna movieId por ID
rati = rati.rename(columns={'movieId': 'ID'})

Ahora vamos a rellenar los datasets uno por uno con los datos faltantes de rating

In [101]:
rati_am = rati[rati['ID'].isin(am['ID'])]
rati_di = rati[rati['ID'].isin(di['ID'])]
rati_hu = rati[rati['ID'].isin(hu['ID'])]
rati_ne = rati[rati['ID'].isin(ne['ID'])]

am = pd.concat([am, rati_am])
di = pd.concat([di, rati_di])
hu = pd.concat([hu, rati_hu])
ne = pd.concat([ne, rati_ne])


In [102]:
#Eliminamos los valores nulos
am = am.dropna(subset=['title'])
di = di.dropna(subset=['title'])
hu = hu.dropna(subset=['title'])
ne = ne.dropna(subset=['title'])

In [103]:
ne.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8807 entries, 0 to 8806
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID            8807 non-null   object 
 1   show_id       8807 non-null   object 
 2   type          8807 non-null   object 
 3   title         8807 non-null   object 
 4   director      6173 non-null   object 
 5   cast          7982 non-null   object 
 6   country       7976 non-null   object 
 7   date_added    8797 non-null   object 
 8   release_year  8807 non-null   float64
 9   rating        8803 non-null   object 
 10  duration      8804 non-null   object 
 11  listed_in     8807 non-null   object 
 12  description   8807 non-null   object 
 13  userId        0 non-null      float64
 14  timestamp     0 non-null      float64
dtypes: float64(3), object(12)
memory usage: 1.1+ MB


## Cambiamos los nulos para la columna rating por 'G'


 Primero analisamos el dataset de Hulu para identificar los valores null

In [104]:
#Vemos los nulos
hu.isnull().sum()


ID                 0
show_id            0
type               0
title              0
director        3070
cast            3073
country         1453
date_added        28
release_year       0
rating           520
duration         479
listed_in          0
description        4
userId          3073
timestamp       3073
dtype: int64

In [105]:
#Transformamos todos los nulos por 'g'
hu['rating'].fillna('G', inplace = True)

In [106]:
#Verificamos que si hallan cambiado
hu.isnull().sum()

ID                 0
show_id            0
type               0
title              0
director        3070
cast            3073
country         1453
date_added        28
release_year       0
rating             0
duration         479
listed_in          0
description        4
userId          3073
timestamp       3073
dtype: int64

In [107]:
#Repetimos para el resto de plataformas
am['rating'].fillna('G', inplace = True)
di['rating'].fillna('G', inplace = True)
ne['rating'].fillna('G', inplace = True)

di.isnull().sum()

ID                 0
show_id            0
type               0
title              0
director         473
cast             190
country          219
date_added         3
release_year       0
rating             0
duration           0
listed_in          0
description        0
userId          1450
timestamp       1450
dtype: int64

## Vamos a cambiar el formato de las fechas por AAA-mm-dd

In [108]:
#Vemos que formato tienen las fechas
am.head()

,ID,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,userId,timestamp
0,as1,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014.0,G,113 min,"Comedy, Drama",A small fishing village must procure a local d...,NaN,NaN
1,as2,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018.0,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...,NaN,NaN
2,as3,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017.0,G,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...,NaN,NaN
3,as4,s4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,"March 30, 2021",2014.0,G,69 min,Documentary,"Pink breaks the mold once again, bringing her ...",NaN,NaN
4,as5,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,"March 30, 2021",1989.0,G,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...,NaN,NaN


In [109]:
#Cambiamos el tipo de formato de fecha primero a Mes dia, año y luego lo pasamos para AAAA-mm-dd mas comodamente
am['date_added'] = pd.to_datetime(am['date_added'], format='%B %d, %Y').dt.strftime('%Y-%m-%d')
di['date_added'] = pd.to_datetime(di['date_added'], format='%B %d, %Y').dt.strftime('%Y-%m-%d')
hu['date_added'] = pd.to_datetime(hu['date_added'], format='%B %d, %Y').dt.strftime('%Y-%m-%d')
#ne['date_added'] = pd.to_datetime(ne['date_added'], format='%B %d, %Y').dt.strftime('%Y-%m-%d')
#ValueError: time data ' August 4, 2017' does not match format '%B %d, %Y' (match)

Notamos que nos tira error al cambiar el formato en netflix, esto se debe porque en la columna 'date_added' tiene algunos espacios y no cumple con el formato .to_datetime()

In [110]:
#Para solucionar el problemas añadimos la funcion .str.strip() para elminar los espacios en blancos
ne['date_added'] = pd.to_datetime(ne['date_added'].str.strip(), format='%B %d, %Y').dt.strftime('%Y-%m-%d')

In [111]:
#verificamos
ne.head()

,ID,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,userId,timestamp
0,ns1,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,2021-09-25,2020.0,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",NaN,NaN
1,ns2,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021-09-24,2021.0,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",NaN,NaN
2,ns3,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,2021-09-24,2021.0,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,NaN,NaN
3,ns4,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,2021-09-24,2021.0,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",NaN,NaN
4,ns5,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,2021-09-24,2021.0,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,NaN,NaN


## Posteriormente transformaremos todos los campos de texto en minusculas
 para esto utilizaremos la funcion lower()

In [112]:
#primero filtramos seleccionando solo las columnas que sean tipo object
#Luego utilizamos la funcion apply() para que una funcion cambie los caracteres
am[am.select_dtypes(include='object').columns] = am.select_dtypes(include='object').apply(lambda x: x.str.lower())
di[di.select_dtypes(include='object').columns] = di.select_dtypes(include='object').apply(lambda x: x.str.lower())
hu[hu.select_dtypes(include='object').columns] = hu.select_dtypes(include='object').apply(lambda x: x.str.lower())
ne[ne.select_dtypes(include='object').columns] = ne.select_dtypes(include='object').apply(lambda x: x.str.lower())

## Y por ultimo crearemos todos columnas de duration para organizar la columna duration

In [113]:
am.head()

,ID,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,userId,timestamp
0,as1,s1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014.0,g,113 min,"comedy, drama",a small fishing village must procure a local d...,NaN,NaN
1,as2,s2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018.0,13+,110 min,"drama, international",a metro family decides to fight a cyber crimin...,NaN,NaN
2,as3,s3,movie,secrets of deception,josh webber,"tom sizemore, lorenzo lamas, robert lasardo, r...",united states,2021-03-30,2017.0,g,74 min,"action, drama, suspense",after a man discovers his wife is cheating on ...,NaN,NaN
3,as4,s4,movie,pink: staying true,sonia anderson,"interviews with: pink, adele, beyoncé, britney...",united states,2021-03-30,2014.0,g,69 min,documentary,"pink breaks the mold once again, bringing her ...",NaN,NaN
4,as5,s5,movie,monster maker,giles foster,"harry dean stanton, kieran o'brien, george cos...",united kingdom,2021-03-30,1989.0,g,45 min,"drama, fantasy",teenage matt banting wants to work with a famo...,NaN,NaN


In [114]:
#Creamos las dos columnas expandiendo los datos de la columan duration
ne[['duration_int','duration_type']] = ne['duration'].str.split(' ', expand=True)

#Convertimos los nulos en 0 y convertimos todos los datos en enteros
ne['duration_int'] = ne['duration_int'].fillna(0).astype(int)
ne['duration_int'] = ne['duration_int'].astype(int)

#Convertimos la otra columna en str en caso que no este asi
ne['duration_type'] = ne['duration_type'].astype(str)

#Por ultimo eliminamos la columna duration
ne.drop(columns=['duration'], inplace=True)

#Repetimos el mismo proceso con el resto de datasets1
am[['duration_int','duration_type']] = am['duration'].str.split(' ', expand=True)
am['duration_int'] = am['duration_int'].fillna(0).astype(int)
am['duration_int'] = am['duration_int'].astype(int)
am['duration_type'] = am['duration_type'].astype(str)
am.drop(columns=['duration'], inplace=True)

hu[['duration_int','duration_type']] = hu['duration'].str.split(' ', expand=True)
hu['duration_int'] = hu['duration_int'].fillna(0).astype(int)
hu['duration_int'] = hu['duration_int'].astype(int)
hu['duration_type'] = hu['duration_type'].astype(str)
hu.drop(columns=['duration'], inplace=True)

di[['duration_int','duration_type']] = di['duration'].str.split(' ', expand=True)
di['duration_int'] = di['duration_int'].fillna(0).astype(int)
di['duration_int'] = di['duration_int'].astype(int)
di['duration_type'] = di['duration_type'].astype(str)
di.drop(columns=['duration'], inplace=True)

In [115]:
am.info(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9668 entries, 0 to 9667
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID             9668 non-null   object 
 1   show_id        9668 non-null   object 
 2   type           9668 non-null   object 
 3   title          9668 non-null   object 
 4   director       7586 non-null   object 
 5   cast           8435 non-null   object 
 6   country        672 non-null    object 
 7   date_added     155 non-null    object 
 8   release_year   9668 non-null   float64
 9   rating         9668 non-null   object 
 10  listed_in      9668 non-null   object 
 11  description    9668 non-null   object 
 12  userId         0 non-null      float64
 13  timestamp      0 non-null      float64
 14  duration_int   9668 non-null   int32  
 15  duration_type  9668 non-null   object 
dtypes: float64(3), int32(1), object(12)
memory usage: 1.2+ MB


## Unificamos todos los datasaets para su mejor manipulacion y creamos un csv con ello

In [116]:
plat = pd.concat([am, di, hu, ne])

plat.to_csv('Data/Plataformas.csv')

In [118]:
plat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22998 entries, 0 to 8806
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID             22998 non-null  object 
 1   show_id        22998 non-null  object 
 2   type           22998 non-null  object 
 3   title          22998 non-null  object 
 4   director       14739 non-null  object 
 5   cast           17677 non-null  object 
 6   country        11499 non-null  object 
 7   date_added     13444 non-null  object 
 8   release_year   22998 non-null  float64
 9   rating         22998 non-null  object 
 10  listed_in      22998 non-null  object 
 11  description    22994 non-null  object 
 12  userId         0 non-null      float64
 13  timestamp      0 non-null      float64
 14  duration_int   22998 non-null  int32  
 15  duration_type  22998 non-null  object 
dtypes: float64(3), int32(1), object(12)
memory usage: 2.9+ MB
